In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score



In [11]:
flight_departures = pd.read_csv('Departing Data.csv')
flight_departures.drop('NewMsgTime',axis=1,inplace=True)
flight_departures.head(30)

,FltNbr,AircraftType,SchedDepTime,SchedArrTime,BlockArrDate,BlockArrTime,MsgDate,MsgTime,NewMsgDate,BlockDepDate,...,VV,N,U,WW,IX,M,R,S,O,Y
0,4,321,930,1045,231112,1049,231112,913,231112,NaN,...,59,8,93,62,7,0,1,0,0,0
1,4,321,930,1045,231112,1049,231112,937,231112,NaN,...,59,8,93,62,7,0,1,0,0,0
2,4,321,1045,1200,231112,1155,231112,1036,231112,NaN,...,61,8,92,57,7,0,1,0,0,0
3,4,321,1045,1200,231112,1155,231112,1046,231112,NaN,...,61,8,92,57,7,0,1,0,0,0
4,4,321,1355,1510,231112,1458,231112,1335,231112,NaN,...,65,8,91,,5,0,0,0,0,0
5,4,321,1355,1510,231112,1458,231112,1352,231112,NaN,...,65,8,91,,5,0,0,0,0,0
6,4,321,1520,1635,231112,1629,231112,1454,231112,NaN,...,70,2,90,,5,0,0,0,0,0
7,4,321,1520,1635,231112,1629,231112,1524,231112,231112.0,...,70,2,90,,5,0,0,0,0,0
8,4,321,1905,2020,231112,2002,231112,1835,231112,231112.0,...,62,1,97,,5,0,0,0,0,0
9,4,321,1905,2020,231112,2002,231112,1902,231112,NaN,...,62,1,97,,5,0,0,0,0,0


In [8]:
"""
mean = flight_departures.mean(axis=0)
meanE = flight_departures['Error'].mean(axis=0)
stdE = flight_departures['Error'].std(axis=0)
flight_depdata = flight_departures - mean
std = flight_departures.std(axis=0)
flight_depdata = flight_departures / std

"""
scaler = MinMaxScaler(feature_range=(0, 1))
flight_departures_scaled = scaler.fit_transform(flight_departures.values[:,1:])
flight_departures.loc[:,1:] = flight_departures_scaled
flight_departures.head()
flight_depdata = flight_departures


X = flight_depdata.drop('Error',axis=1)
y = flight_depdata['Error']
x = X.as_matrix()
y = y.as_matrix()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

ValueError: could not convert string to float: 'E70'

In [ ]:
flight_depdata.shape

In [ ]:
x_val = x_train[:20000]
partial_x_train = x_train[20000:]

y_val = y_train[:20000]
partial_y_train = y_train[20000:]

In [ ]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(128,  
                           activation='relu',
                           input_shape=(x_train.shape[1],)))
    model.add(layers.Dense(128, 
                           activation='relu'))
    model.add(layers.Dense(256,
                           activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
model = build_model()
history = model.fit(partial_x_train, partial_y_train,
                   validation_data=(x_val, y_val),
                   epochs=100, batch_size=128, verbose=0)
val_mae = history.history['val_mean_absolute_error']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.show()

In [ ]:
mae_history = history.history['val_mean_absolute_error']

plt.plot(range(1, len(mae_history) + 1), mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')

plt.show()

In [ ]:
dep_model = build_model()
dep_model.fit(x_train, y_train,
         epochs=50, batch_size=128, verbose=0)

results = dep_model.evaluate(x_test, y_test)
predictions = dep_model.predict(x_test)
r2 = r2_score(y_test,predictions)

In [ ]:
results

In [ ]:
print(r2)

In [ ]:
predicted_df = pd.DataFrame(predictions,columns=['Predicted Error'])

In [ ]:
unstand_predict = (predicted_df * stdE) + meanE
unstand_predict.head()

In [ ]:
test_df = pd.DataFrame(y_test,columns=['True Error'])
unstand_test = (test_df * stdE) + meanE
unstand_test.head()

In [ ]:
# plt.plot(range(len(predicted_df)), unstand_predict['Predicted Error'], 'bo', label='Predictions')
# plt.plot(range(len(test_df)), unstand_test['True Error'], 'ro', label='True Error')
# plt.title('Predictions with True values')
# plt.xlabel('Index')
# plt.ylabel('Error')

# plt.show()

In [ ]:
print(history.history.keys())